In [1]:
import pandas as pd
import yfinance as yf
from matplotlib import pyplot

In [2]:
tsla = yf.Ticker('TSLA')
tsla_data = tsla.history(period='2y')
tsla_data.reset_index(inplace=True)

In [11]:
open_price = tsla_data.iloc[:,1:2].values

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(open_price)

array([[0.04491859],
       [0.03414999],
       [0.05701982],
       [0.03312668],
       [0.0209779 ],
       [0.03541128],
       [0.02674884],
       [0.01899076],
       [0.01534966],
       [0.        ],
       [0.01871705],
       [0.04357396],
       [0.04081342],
       [0.05009462],
       [0.05414026],
       [0.03104438],
       [0.02671317],
       [0.02504734],
       [0.02004974],
       [0.01776515],
       [0.07873539],
       [0.06467082],
       [0.11660977],
       [0.12373726],
       [0.13029354],
       [0.17420069],
       [0.18569509],
       [0.22270084],
       [0.24774813],
       [0.24225083],
       [0.19328661],
       [0.23324333],
       [0.23442128],
       [0.25097275],
       [0.27547267],
       [0.30900395],
       [0.21462144],
       [0.26297877],
       [0.26772646],
       [0.29661714],
       [0.27870923],
       [0.27866162],
       [0.32710223],
       [0.32393714],
       [0.30223345],
       [0.28343311],
       [0.29627208],
       [0.333